In [2]:
import pandas as pd
import numpy as np

In [3]:
LOC = "~/Work/Data/FondAsp"
df = pd.read_csv(f"{LOC}/results.csv")
df.head()

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
0,acrobatics,p06,asp1-4h,asp,False,-1,14400.383668,2158.449219,True,False,36
1,acrobatics,p06,paladinus-4h,paladinus,True,True,3.153158,82.828125,False,False,128
2,acrobatics,p06,minisat-4h,fondsat,False,-1,14400.285015,1983.265625,True,False,-1
3,acrobatics,p06,glucose-4h,fondsat,False,-1,14399.936225,610.523438,True,False,-1
4,acrobatics,p06,asp2-backbone-4h,asp,False,-1,14400.779252,2543.703125,True,False,38


## Preprocess: Let us separate SAT from UNSAT instances

In [4]:
paladinus_incorrect = ["tireworld-truck", "earth_observation"] ## Paladinus reports these incorrectly
df_unsat_instances = df.query(f"solved==True and SAT=='False' and planner=='paladinus-4h' and domain not in @paladinus_incorrect")
df_unsat_instances[["domain", "instance"]].head()

,domain,instance
1019,tireworld,p01
1049,tireworld,p09
1067,tireworld,p15
2717,first-responders-ipc08,p50
2724,first-responders-ipc08,p20


In [5]:
# Save the SAT and UNSAT instances so that next time we don't need to do this step again!
df_sat = df[~(df.domain.isin(df_unsat_instances.domain) & df.instance.isin(df_unsat_instances.instance))]
df_sat.loc[df_sat.SAT=='False', "solved"] = False
df_sat.to_csv(f"{LOC}/sat_results.csv")

df_unsat = df[(df.domain.isin(df_unsat_instances.domain) & df.instance.isin(df_unsat_instances.instance))]
df_unsat.to_csv(f"{LOC}/unsat_results.csv")

### Count the total number of instances and store them

In [6]:
df_sat_counts = df_sat.query("planner=='paladinus-4h'").groupby(["domain"]).count()[["instance"]]
df_sat_counts.rename(columns={"instance": "total_count"}, inplace=True)
df_sat_counts.to_csv(f"{LOC}/counts.csv")
df_sat_counts

,total_count
domain,
acrobatics,8
beam-walk,11
blocksworld-ipc08,30
blocksworld-new,40
chain-of-rooms,10
doors,15
earth_observation,40
elevators,15
faults-ipc08,55
